比起物件導向  
Python 在許多方面更像是結構化或函式性程式設計語言  
使用 Python 時  
大部分工具已是底層物件導向實作的語法糖  
我們在這一章會討論 Python 中非嚴格的物件導向特質  
* 以一個呼叫處理常見任務的內建函式
* 檔案 I/O 與背景處理
* 另一種方法過載
* 函式做為物件

# Python 內建函式
---
Python 中有許多函式對特定物件型別執行任務或計算結果  
但並非是底層物件上的方法  
  
他們通常是可套用於多種類別的抽象通用計算  
這是一種鴨子型別  
這些函式有特定屬性或方法的物件並能夠使用這些含是執行共通操作  
他們有許多但非全部都是雙底線方法  
來快速討論幾個重要的函式

## `len()`
---
用來計算字典或清單中物件的數量

In [1]:
len([1,2,3,4])

4

這些物件其實有 `__len__()` 方法來計算數量

In [2]:
[1,2,3,4].__len__()

4

那為什麼我們要用 `len()` 呢  
很明顯的 `__len__()` 是個特殊的雙底線方法  
建議不要直接呼叫他  
    
主要原因在於效率  
當我們呼叫物件的 `__len()__` 時  
物件必須在他的命名空間中查詢方法  
如果物件有定義特殊的 `__getattribute__` 方法  
他就會被呼叫，或是拒絕存取  
而 `len()` 不會遇到這種情況  
他實際上是呼叫底層類別 `__len__`  
因此 `len(myobj) = MyObj.__len__(myobj)`  

另一個原因在於效率  
若 Python 開發者未來想改變 `len()`  
讓他可以逐個計算沒有 `__len__` 的物件長度  
只需要改變 `len()` 就好而不用改變所有的 `__len__`

## 反轉
---
`reversed()` 將任何順序的輸入回傳一份反向順序的輸出  
類似於 `len`  
`reversed()` 呼叫類別上的 `__reversed__()`  
如果不存在這個方法  
會自行使用 `__len__` 和 `__getitem__` 來建構反向順序

In [3]:
normal_list = [1,2,3,4,5]

class CustomSequence():
    def __len__(self):
        return 5
    def __getitem__(self, index):
        return f'x{index}'

class FunkyBackwards():
    def __reversed__(self):
        return "BACKWARDS!"

for seq in normal_list, CustomSequence(), FunkyBackwards():
    print(f"\n{seq.__class__.__name__}: ", end = "")
    for item in reversed(seq):
        print(item, end= ", ")




list: 5, 4, 3, 2, 1, 
CustomSequence: x4, x3, x2, x1, x0, 
FunkyBackwards: B, A, C, K, W, A, R, D, S, !, 

## 列舉
---
`for` 迴圈不提供索引 (index)  
不過 `enumerate` 可以

In [ ]:
import sys
filename = sys.argv[1]

with open(filename) as file:
    for index, line in enumerate(file):
        print(f'{index+1}: {line}')


如果將上述指令碼變成一個文件檔丟進終端機中  
會呈現如下結果  
  
1: import sys  
2: filename = sys.argv[1]  
3:   
4: with open(filename) as file:  
5: for index, line in enumerate(file):  
6: print(f'{index+1}: {line}')  

還有很多常用的標準函式庫  
* `all()`  
* `any()`  
* `eval()`
* `exec()`
* `compile()`
* `hasattr()`
* `getattr()`
* `setattr()`
* `delattr()`
* `zip()`

## 檔案 I/O
---
內建的 `open()` 用於開啟檔案並回傳檔案物件  
當然  
我們不總是只有讀取檔案  
也要開啟檔案寫入  
`open()` 的第二個參數 `mode` 傳入 `w`

In [5]:
contents = "some file contents"
file = open("filename", 'w')
file.write(contents)
file.close()

在 `open()` 中也可以提供 `a` 給 `mode`  
可將內容新增在原有內容後面  
  
要開啟二進位檔  
則在 `mode` 傳入的參數後面加上 `b`   
例如 `wb` 則是寫入二進位  
`rb` 則是讀取二進位  
  
檔案開啟供讀取之後  
可以呼叫 `read`、`readline`、`readlines` 方法來取得檔案內容  
`read` 可以指定一次要讀取多少位元組  
`readline` 則是一行一行讀取  
`readlines` 則是全部讀取並以清單顯示

## 置入背景
---
在任何 I/O 都有可能拋出例外  
對每個 I/O 都要寫上 `try...finally` 很不python  
如果我們對檔案物件執行 `dir`
我們會看到他有 `__enter__` 和 `__exit__` 這兩個特殊方法  
這些方法讓檔案物件轉換成**背景管理員(context manager)**  
使用 `with` 呼叫時  
這些方法會在被套疊程式碼執行的前後呼叫  
也確保檔案就算有例外出現時也會被關閉  

In [6]:
with open('filename') as file:
    for line in file:
        print(line, end='')

some file contents

我們也能自定義 `__enter__` 和 `__exit__` 的行為  


In [7]:
class StringJoiner(list):
    def __enter__(self):
        return self
    
    def __exit__(self, type, value, tb):
        self.result = "".join(self)

import random, string
with StringJoiner() as joiner:
    for i in range(15):
        joiner.append(random.choice(string.ascii_letters))

print(joiner.result)

jaFFpRwdNMjgfFo


此程式建構 15 個隨機字元組成的字串  
繼承自 `list` 使用 `append` 方法將字元插入到 `StringJoiner` 中  
當 `with` 的縮排結束  
`__exit__` 被呼叫  
此時物件的 `result` 屬性就緒  

# 方法過載替代方案
---

## 預設參數
---
我們可以使用等號在單一方法中指派預設值  
且預設值一定要放在沒有預設值的參數的後面

In [8]:
def default_arguments(x, y, z, a='some string', b=False):
    pass

前三個參數還是必須由呼叫方傳入  
我們可以依序傳入參數  
也可以只傳入前三個  
也可以利用等號混合排列傳入參數  

使用關鍵字參數要注意的一件事是我們所提供的預測參數在函式被解譯是會先求值  
並非等到呼叫時

In [9]:
number = 5
def funky_function(number=number):
    print(number)

number = 6
funky_function(8)
funky_function()
print(number)

8
5
6


上述第 6 行確實把 8 帶進函式中  
第 7 行則沒有因為 number 代入 6 而輸出 6  
而是輸出早先定義的 5  
這邊要特別注意  
  
當參數為清單集合字典時也會有意想不到的結果

In [10]:
def hello(b=[]):
    b.append('a')
    print(b)

In [11]:
hello()

['a']


In [12]:
hello()

['a', 'a']


第二次呼叫 `hello()` 居然不是先創建一個空清單再插入 a  

## 變數參數清單
---
光靠預設值不能完全享受方法過載的好處  
讓 Python 真正棒的是  
在不用明確的命名下，就能夠撰寫接受不定數量的位置或關鍵字參數

In [13]:
def get_pages(*links):
    for link in links:
        # 使用urllib下載連結
        print(link)

get_pages()
get_pages('http://www.archlinux.org')
get_pages('http://www.archlinux.org','http://ccphillips.net/')

http://www.archlinux.org
http://www.archlinux.org
http://ccphillips.net/


Python 也可以接受不定量的關鍵字參數  
他們會在函式中成為字典

In [14]:
class Options:
    default_options = {
        'port': 21,
        'host': 'localhost',
        'username': None,
        'password': None,
        'debug': False,
    }
    
    def __init__(self, **kwargs):
        self.options = dict(Options.default_options)
        self.options.update(kwargs)
    
    def __getitem__(self, key):
        return self.options[key]

options = Options(username = "dusty", password="drowssap", debug=True)
print(options['debug'])
print(options['port'])
print(options['username'])

True
21
dusty


關鍵字參數有可能打破〝明確比隱含好〞的規則  
在前面的例子中  
使用者可隨意增加參數到 Options 初始化程序中  
雖然不是什麼壞事  
但有可能讓使用此類別的人很難發現有哪些選項可用  
也很容易打錯字造成應該只有一個選項卻出現兩個(例如 debug 與 Debug)

In [15]:
import shutil
import os.path

def augmented_move(target_folder, *filenames, verbose=False, **specific):
    '''
    將所有檔名移入 target_folder 已執行特定處理
    '''
    def print_verbose(message, filename):
        '''僅於verbose = True 時輸出訊息'''
        if verbose:
            print(message.format(filename))
    
    for filename in filenames:
        target_path = os.path.join(target_folder, filename)
        if filename in specific:
            if specific[filename] == 'ignore':
                print_verbose(f"Ingoring {filename}")
            elif specific[filename] == 'copy':
                print_verbose(f'Copying {filename}')
                shutil.copyfile(filename, target_path)
        else:
            print_verbose(f'Moving {filename}')
            shutil.move(filename, target_path)

## 展開參數
---
還有一個技巧涉及變數參數與關鍵字參數

In [16]:
def show_args(arg1, arg2, arg3="three"):
    print(arg1, arg2, arg3)

some_args = range(3)
more_args = {
    "arg1": "ONE",
    "arg2": "TWO",
}

print("Unpacking a sequence: ", end=" ")
show_args(*some_args)

print("Unpacking a dict: ", end= " ")
show_args(**more_args)

Unpacking a sequence:  0 1 2
Unpacking a dict:  ONE TWO three


此函式接受三個變數，其中一個為預設值  
當我們有三個參數的清單時  
可以使用 * 運算元將其展開成三個參數丟進函式中  
如果我們有參數字典，可以使用 ** 語法將其展開作為關鍵字參數的集合  

# 函式也是物件
---
在Python中  
我們可以設置函式的屬性(這不是常見的動作)  
可以傳遞他們並於之後呼叫

In [18]:
def my_function():
    print("The Function Was Called")
my_function.description = "A silly function"

def second_function():
    print("The second was called")
second_function.description = "A sillier function"

def another_function(function):
    print(f"The description: {function.description}")
    print(f"The name: {function.__name__}")
    print(f"The class: {function.__class__}")
    print("Now I'll call the function passed in")
    function()
    print("-------------------------------------")

another_function(my_function)
another_function(second_function)






The description: A silly function
The name: my_function
The class: <class 'function'>
Now I'll call the function passed in
The Function Was Called
-------------------------------------
The description: A sillier function
The name: second_function
The class: <class 'function'>
Now I'll call the function passed in
The second was called
-------------------------------------


## 使用函式作為屬性
---
函式作為物件其中最有意思的效果是他們可以被設定成其他物件上的可呼叫屬性

In [19]:
class A:
    def print(self):
        print("my class is A")
    
def fake_print():
    print("my class in not A")

a = A()
a.print()
a.print = fake_print
a.print()


my class is A
my class in not A
